In [2]:
import matplotlib.pyplot as plt
import numpy as np

# I'm using seaborn for it's fantastic default styles
import seaborn as sns
sns.set_style("whitegrid")

%matplotlib inline
%load_ext autoreload

%autoreload 2

# Es un archivo con funciones útiles
from tutils import BaseStateSystem

In [4]:
def laplacian1D(a, dx):
    return (
        - 2 * a
        + np.roll(a,1,axis=0) 
        + np.roll(a,-1,axis=0)
    ) / (dx ** 2)

def laplacian2D(a, dx):
    return (
        - 4 * a
        + np.roll(a,1,axis=0) 
        + np.roll(a,-1,axis=0)
        + np.roll(a,+1,axis=1)
        + np.roll(a,-1,axis=1)
    ) / (dx ** 2)

In [5]:
class TwoDimensionalRDEquations(BaseStateSystem):
    def __init__(self, P, Ra, Rb,
                 initialiser=random_initialiser,
                 width=1000, height=1000,
                 dx=1, dt=0.1, steps=1):
        
        self.P = P
        self.Ra = Ra
        self.Rb = Rb

        self.initialiser = initialiser
        self.width = width
        self.height = height
        self.shape = (width, height)
        self.dx = dx
        self.dt = dt
        self.steps = steps
        
    def initialise(self):
        self.t = 0
        self.a, self.b = self.initialiser(self.shape)
        
    def update(self):
        for _ in range(self.steps):
            self.t += self.dt
            self._update()

    def _update(self):
        
        # unpack so we don't have to keep writing "self"
        a,b,P,Ra,Rb,dt,dx = (
            self.a, self.b,
            self.P,
            self.Ra, self.Rb,
            self.dt, self.dx
        )
        
        La = laplacian2D(a, dx)
        Lb = laplacian2D(b, dx)
        
        delta_a = dt * (1 * La + Ra(a,b))
        delta_b = dt * (P * Lb + Rb(a,b))
        
        self.a += delta_a
        self.b += delta_b
        
    def draw(self, ax):
        ax[0].clear()
        ax[1].clear()

        ax[0].imshow(self.a, cmap='jet')
        ax[1].imshow(self.b, cmap='brg')
        
        ax[0].grid(False)
        ax[1].grid(False)
        
        ax[0].set_title("A, t = {:.2f}".format(self.t))
        ax[1].set_title("B, t = {:.2f}".format(self.t))
        
    def initialise_figure(self):
        fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12,6))
        return fig, ax



### Único punto random:

In [146]:
def random_initialiser(shape):
    return(
        np.random.normal(loc=0, scale=0.1, size=shape),
        np.random.normal(loc=0, scale=0.1, size=shape)
    )

P, Q, R = 100, 10, 1

def Ra(a,b): return 1 + R*(a**2)/b - a
def Rb(a,b): return Q*(a**2 - b)
width = 200
dx = 1
dt = 0.001

TwoDimensionalRDEquations(
    P, Ra, Rb, 
    width=width, height=width, 
    dx=dx, dt=dt, steps=250
).plot_evolution_outcome("2dRD.png", n_steps=10)

### Con simetrías 3, 5 y 7:

In [150]:
from scipy.ndimage import rotate

def average_rotate(a, degree):
    """
    Takes a 2d array a, and produces the average arrays,
    having rotated it degree times. The resulting shape
    has approximate degree-fold rotational symmetry.
    """
    theta = 360 / degree

    a = np.mean([rotate(a, theta * i, reshape=False) for i in range(degree)], axis=0)

    return a


def random_symmetric_initialiser(shape, degree):
    """
    Random initialiser with degree-fold symmetry.
    """
    
    a = np.random.normal(loc=0, scale=0.05, size=shape)
    b = np.random.normal(loc=0, scale=0.05, size=shape)

    return (
        average_rotate(a, degree), 
        average_rotate(b, degree)
    )

P, Q, R = 3.8, 0.06, 0.062

def Ra(a,b): return 1 + R*(a**2)/b - a
def Rb(a,b): return Q*(a**2 - b)
width = 200
dx = 1
dt = 0.001

# three fold
three_fold_initialiser = lambda shape: random_symmetric_initialiser(shape, 3)

TwoDimensionalRDEquations(
    P, Ra, Rb,  
    initialiser=three_fold_initialiser, 
    width=width, height=width, 
    dx=dx, dt=dt, steps=250
).plot_time_evolution("2dRD_3_fold_sym.gif", n_steps=5)

# five fold
five_fold_initialiser = lambda shape: random_symmetric_initialiser(shape, 5)

TwoDimensionalRDEquations(
    P, Ra, Rb, 
    initialiser=five_fold_initialiser, 
    width=width, height=width, 
    dx=dx, dt=dt, steps=250
).plot_time_evolution("2dRD_5_fold_sym.gif", n_steps=5)

# seven fold
seven_fold_initialiser = lambda shape: random_symmetric_initialiser(shape, 7)

TwoDimensionalRDEquations(
    P, Ra, Rb, 
    initialiser=seven_fold_initialiser, 
    width=width, height=width, 
    dx=dx, dt=dt, steps=250
).plot_time_evolution("2dRD_7_fold_sym.gif", n_steps=5)

MovieWriter imagemagick unavailable; using Pillow instead.
MovieWriter imagemagick unavailable; using Pillow instead.
MovieWriter imagemagick unavailable; using Pillow instead.


### Puntos con simetría equiespaciada cuadrículada:

In [161]:
def eqspace_initialiser(shape):

    a = np.zeros(shape)
    b = np.zeros(shape)
    
    c = 25 #To change the grid size

    m = np.random.normal(loc=0, scale=0.05, size=(c,c))
    n = np.random.normal(loc=0, scale=0.05, size=(c,c))

    for k in range (0,c):
        for h in range (0,c):
            for i in range(0,shape[0],c):
                for j in range(0,shape[0],c):
                    a[i+k][j+h] = m[k][h]
                    b[i+k][j+h] = n[k][h]

    return(a,b)

P, Q, R = 100, 0.06, 0.062

def Ra(a,b): return 1 + R*(a**2)/b - a
def Rb(a,b): return Q*(a**2 - b)
width = 200
dx = 1
dt = 0.001

TwoDimensionalRDEquations(
    P, Ra, Rb, 
    initialiser=eqspace_initialiser,
    width=width, height=width, 
    dx=dx, dt=dt, steps=600
).plot_time_evolution("2dRD_equispacesym.gif", n_steps=150)

MovieWriter imagemagick unavailable; using Pillow instead.


### Barras:

In [148]:
def bar_initialiser(shape):

    a = np.zeros(shape)
    b = np.zeros(shape)

    c = 100 #To change the grid size
    
    m = np.random.normal(loc=0, scale=0.05, size=(1,c))
    n = np.random.normal(loc=0, scale=0.05, size=(1,c))

    for h in range (0,c):
        for i in range(0,shape[0]):
            for j in range(0,shape[0],c):
                a[i][j+h] = m[0][h]
                b[i][j+h] = n[0][h]

    return(a,b)

P, Q, R = 100, 10, 1

def Ra(a,b): return 1 + R*(a**2)/b - a
def Rb(a,b): return Q*(a**2 - b)
width = 200
dx = 1
dt = 0.001

TwoDimensionalRDEquations(
    P, Ra, Rb, 
    initialiser=bar_initialiser,
    width=width, height=width, 
    dx=dx, dt=dt, steps=200
).plot_time_evolution("2dRD_barsym.gif", n_steps=10)

MovieWriter imagemagick unavailable; using Pillow instead.
